# OvR

This notebook tries to create a baseline for the task. Noteworthy points:

-Using LRAP loss metric in cross-validation in Model 2. 

-Best score is around 0.3 in Model 2. It seems low but I feel this task is quite difficult so maybe very high scores are difficult.

-Balancing cross-validation using n_jobs and pre_dispatch in order to preserve maximum CPU usage but prevent memory from blowing up.

In [1]:
import pandas as pd
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.metrics import label_ranking_average_precision_score,make_scorer

# Download data

In [2]:
#Load the previously transformed data.
NUM_FEATURES = 5000
NUM_CLASSES = 3993

x_train = pd.read_csv("../data/expanded/train_features.csv", names=range(NUM_FEATURES))
y_train = pd.read_csv("../data/expanded/train_labels.csv", names=range(NUM_CLASSES))

x_val = pd.read_csv("../data/expanded/dev_features.csv", names=range(NUM_FEATURES))
y_val = pd.read_csv("../data/expanded/dev_labels.csv", names=range(NUM_CLASSES))

In [3]:
x_train.shape

(15511, 5000)

# Model 1

In [4]:
#Training takes around 30min on my machine. 5000 classes  = 5000 seperate models in one-vs-rest scheme so not too efficient.
model = OneVsRestClassifier(LinearSVC(tol = np.exp(-4), random_state=0, dual = False),n_jobs = -1).fit(x_train,y_train)

In [5]:
y_pred = model.predict(x_val)

In [6]:
score = label_ranking_average_precision_score(y_val,y_pred)
score

0.3391564618701288